# Preprocessing Step (Etapa de Pré-Processamento)

## Table of Contents
* [Packages](#1)
* [Preprocessing](#2)

<a name="1"></a>
## Packages (Pacotes)

In [293]:
import pandas as pd
import string
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

<a name="2"></a>
## Preprocessing (Pré-processamento)

In [277]:
comics_data = pd.read_csv('../data/raw/comics_corpus.csv').drop_duplicates()
char_data = pd.read_csv('../data/raw/char_corpus.csv')
comics_data.head()

,id,title,description,y
0,94799,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...,suspense
1,93339,The Mighty Valkyries (2021) #3,CHILDREN OF THE AFTERLIFE! While Kraven the Hu...,action
2,94884,The Mighty Valkyries (2021) #3 (Variant),CHILDREN OF THE AFTERLIFE! While Kraven the Hu...,action
3,93350,X-Corp (2021) #2,A SHARK IN THE WATER! After X-CORP’s shocking ...,suspense
4,94896,X-Corp (2021) #2 (Variant),A SHARK IN THE WATER! After X-CORP?s shocking ...,suspense


In [279]:
char_data.head()

,id,name,description,y
0,1017100,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...,action
1,1009144,A.I.M.,AIM is a terrorist organization bent on destro...,action
2,1009146,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie...",action
3,1010354,Adam Warlock,Adam Warlock is an artificially created human ...,action
4,1011031,Agent X (Nijo),Originally a partner of the mind-altering assa...,action


In [281]:
comics_data['description'].str.contains('!').sum()

np.int64(12945)

In [283]:
MAX_TOKENS = None
MAX_LEN = max([len(text) for text in comics_data['description']])
MAX_LEN

1503

In [287]:
def rnn_preprocess(corpus):
    """
    
    """
    stem = PorterStemmer()
    stopwords_en = stopwords.words('english')
    
    corpus = re.sub(f'r[{re.escape(string.punctuation)}]', '.', corpus)
    corpus = re.sub(f'r{stopwords_en}', '', corpus)
    corpus = re.sub(r'\s+', ' ', corpus)
    corpus = re.sub(r'\W+', ' ', corpus).strip().lower()   
    
    return corpus

In [289]:
comics_data['description'].map(rnn_preprocess)

0        in the shadow of kirisaki mountain a secret hi...
1        children of the afterlife while kven the hunte...
2        children of the afterlife while kven the hunte...
3        a shark in the water aftex corp s shocking deb...
4        a shark in the water aftex corp s shocking deb...
                               ...                        
16922    a collection of epic stoes featung human toh i...
16923    d jack castle investigates a sees of mue and c...
16924    a histoc comic sees dating all the way back to...
16925    the one that staed it all featung the fit appe...
16926    concluding this chilling chapter of stephen ki...
Name: description, Length: 16927, dtype: object

In [182]:
comics_data['description'] = comics_data['description'].map(rnn_preprocess)
comics_data.head()

,id,title,description,y
0,94799,Demon Days: Mariko (2021) #1 (Variant),in the shadow of kirisaki mountain a secret hi...,suspense
1,93339,The Mighty Valkyries (2021) #3,children of the afterlife while kraven the hun...,action
2,94884,The Mighty Valkyries (2021) #3 (Variant),children of the afterlife while kraven the hun...,action
3,93350,X-Corp (2021) #2,a shark in the water after x corp s shocking d...,suspense
4,94896,X-Corp (2021) #2 (Variant),a shark in the water after x corp s shocking d...,suspense


In [80]:
comics_data['description'].str.contains('!').sum()

np.int64(0)

In [212]:
def rnn_vectorization(corpus, max_tokens=None):
    """
    
    """
    sentence_vec = TextVectorization(
        standardize='lower_and_strip_punctuation',
        max_tokens=max_tokens,
        ragged=True
    )
    sentence_vec.adapt(corpus)
    vocabulary = sentence_vec.get_vocabulary(include_special_tokens=True)

    return sentence_vec, vocabulary

In [256]:
sentence_vec, vocabulary = rnn_vectorization(comics_data['description'], max_tokens=None)
len(vocabulary)

23337

In [216]:
X_vec = sentence_vec(comics_data['description'])

In [244]:
def padding(corpus, maxlen=None, shuffle_buffer_size=1000, prefetch_buffer_size=tf.data.AUTOTUNE, batch_size=128):
    """
    
    """
    padded_corpus = tf.keras.utils.pad_sequences(
        corpus.numpy(),
        maxlen=maxlen,
        truncating='post',
        padding='post'
    )
    padded_corpus = (
        tf.data.Dataset.from_tensor_slices(padded_corpus)
        .cache()
        .shuffle(shuffle_buffer_size)
        .prefetch(prefetch_buffer_size)
        .batch(batch_size)
    )

    return padded_corpus

In [260]:
X_pad = padding(X_vec, maxlen=MAX_LEN)

In [262]:
next(iter(X_pad.take(1)))

<tf.Tensor: shape=(128, 1468), dtype=int32, numpy=
array([[   2, 9203,    3, ...,    0,    0,    0],
       [ 426,    2,  190, ...,    0,    0,    0],
       [  85,  108,   75, ...,    0,    0,    0],
       ...,
       [  13,   84,   32, ...,    0,    0,    0],
       [ 171,  187,   22, ...,    0,    0,    0],
       [1560,   40,    3, ...,    0,    0,    0]], dtype=int32)>

In [264]:
y_dataset = tf.data.Dataset.from_tensor_slices(comics_data['y'])
comics_dataset = tf.data.Dataset.zip(X_pad, y_dataset)
next(iter(comics_dataset.take(1)))

(<tf.Tensor: shape=(128, 1468), dtype=int32, numpy=
 array([[ 810,   52,  352, ...,    0,    0,    0],
        [1742,    2, 7480, ...,    0,    0,    0],
        [  24,   21,  881, ...,    0,    0,    0],
        ...,
        [   6,   21,  101, ...,    0,    0,    0],
        [  23,  717,    7, ...,    0,    0,    0],
        [  20,   80,   33, ...,    0,    0,    0]], dtype=int32)>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'suspense'>)

In [82]:
punctuations = string.punctuation
stopwords_en = stopwords.words('english')
print(f'Punctuations: {punctuations}\n\nStopwords: {stopwords_en}')

Punctuations: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

Stopwords: ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such',

In [110]:
re.escape(' '.join(stopwords_en))

"i\\ me\\ my\\ myself\\ we\\ our\\ ours\\ ourselves\\ you\\ you're\\ you've\\ you'll\\ you'd\\ your\\ yours\\ yourself\\ yourselves\\ he\\ him\\ his\\ himself\\ she\\ she's\\ her\\ hers\\ herself\\ it\\ it's\\ its\\ itself\\ they\\ them\\ their\\ theirs\\ themselves\\ what\\ which\\ who\\ whom\\ this\\ that\\ that'll\\ these\\ those\\ am\\ is\\ are\\ was\\ were\\ be\\ been\\ being\\ have\\ has\\ had\\ having\\ do\\ does\\ did\\ doing\\ a\\ an\\ the\\ and\\ but\\ if\\ or\\ because\\ as\\ until\\ while\\ of\\ at\\ by\\ for\\ with\\ about\\ against\\ between\\ into\\ through\\ during\\ before\\ after\\ above\\ below\\ to\\ from\\ up\\ down\\ in\\ out\\ on\\ off\\ over\\ under\\ again\\ further\\ then\\ once\\ here\\ there\\ when\\ where\\ why\\ how\\ all\\ any\\ both\\ each\\ few\\ more\\ most\\ other\\ some\\ such\\ no\\ nor\\ not\\ only\\ own\\ same\\ so\\ than\\ too\\ very\\ s\\ t\\ can\\ will\\ just\\ don\\ don't\\ should\\ should've\\ now\\ d\\ ll\\ m\\ o\\ re\\ ve\\ y\\ ain\\ aren\\